In [83]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
import plotly.graph_objects as go
import plotly.io as pio

# Configurar Plotly para que funcione en Jupyter Notebook
pio.renderers.default = 'notebook'

In [84]:
# Leer y preprocesar datos (igual que antes)
file_path = 'Evolucion_IPC_esp.xlsx'
df = pd.read_excel(file_path)
df.columns = df.columns.str.strip()
df['Periodo'] = pd.to_datetime(df['Periodo'], format='%d/%m/%y', errors='coerce')
df.set_index('Periodo', inplace=True)

columna = 'Bebidas alcohólicas y tabaco'
serie_temporal = df[columna]

In [85]:
# Normalización de los datos
scaler = MinMaxScaler(feature_range=(0, 1))
serie_temporal_scaled = scaler.fit_transform(serie_temporal.values.reshape(-1, 1))

In [86]:
# Preparar datos para LSTM
ventana = 12
X, y = [], []
for i in range(len(serie_temporal_scaled) - ventana):
    X.append(serie_temporal_scaled[i:i + ventana])
    y.append(serie_temporal_scaled[i + ventana])

X, y = np.array(X), np.array(y)

In [87]:
# Definir el modelo LSTM
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(ventana, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [88]:
# Entrenar el modelo
model.fit(X, y, epochs=20, batch_size=16, verbose=0)

In [101]:
# Generar predicciones futuras
future_steps = 31
input_data = serie_temporal_scaled[-ventana:].reshape(1, ventana, 1)
futuras_predicciones = []

for _ in range(future_steps):
    pred_scaled = model.predict(input_data)
    futuras_predicciones.append(pred_scaled[0, 0])
    new_input = np.array(pred_scaled[0, 0]).reshape(1, 1, 1)
    input_data = np.append(input_data[:, 1:, :], new_input, axis=1)

futuras_predicciones = scaler.inverse_transform(np.array(futuras_predicciones).reshape(-1, 1))

1/1 [==============================] - 0s 38ms/step


In [102]:
# Generar el índice de tiempo para las predicciones futuras
future_index = pd.date_range(start=serie_temporal.index[-1], periods=future_steps + 1, freq='MS')[1:]

In [107]:
# Crear una gráfica interactiva con Plotly
fig = go.Figure()

In [108]:
# Datos históricos
fig.add_trace(go.Scatter(
    x=serie_temporal.index,
    y=serie_temporal.values,
    mode='lines',
    name='Datos históricos',
    line=dict(color='blue')
))

In [114]:
# Predicciones futuras
fig.add_trace(go.Scatter(
    x=future_index,
    y=futuras_predicciones.flatten(),
    mode='lines',
    name='Predicción futura',
    line=dict(color='red', dash='dot')
))

In [111]:
# Personalizar el diseño
fig.update_layout(
    title=f'Predicción LSTM para: {columna}',
    xaxis_title='Fecha',
    yaxis_title='Valor',
    legend=dict(x=0, y=1, bgcolor='rgba(255, 255, 255, 0)', bordercolor='rgba(0, 0, 0, 0)'),
    template='plotly_white'
)